# Unit commitment Formulation 

Unit commitment(UC) problem is a fundamental nonlinear non-convex NP-hard optimization problem which makes finding optimal solution difficult. The problem is usually modeled as a mixed-integer programming. The objective of UC problem is to achieve a common target given a set of generating units  subject to  a set of constraints.

The importance of the problem is reflected on the large different approaches suggested to solve the problem which requires requires fast decisions. Quantum computers may provide and advantage in this case.


## Problem Statement

The problem of Unit Commitment deals with what generators should be turned on/off and when depending on the demand so as to minimize costs.

 The setup is as follows. We have a power grid with various power generation units(nodes) that are connected by transmission lines. For simplicty, we assume that we have a set of nodes with one load. The transmion costs is identical(no edges in this case). To simplify the problem furhter, we nly consider one time step. Each node has a set of physical characterisctis: 
 
 A: fixed cost of the unit when it is turned on regardless of the power level.
 
 B: linear cost based on the power level.
 
 C: quandratic cost based on the power level.
 
 Total cost per unit: $C(P) = A + B*P + C* P^2$ where $P$ is the allocatated power to the unit. Classical methods are used to find this paparmeter like economic dispatch.  The total cost is  the sum of the costs of the units that are on.

 
 
 $P_{min}$: minimum generation limit.
 
 $P_{max}$: maximum generation limit.
 
There are many other operational parameters that can be added easily. For simplicity, we consider these. Fig. 1 demonstrates two ways that the load of a 4-unit system could be met while keeping within each unit’s
constraints, and gives the cost of each configuration [[1]](https://arxiv.org/abs/2110.12624).


![alt text](uc.jpg "ref[]")



Thus the  variant we are dealing with can be formulated as the following binary problem:

$$
\begin{aligned}
 min & \sum_{i=1}^{n} y_i C_i(P) = \sum_{i=1}^{n}(y_i(A_i + B_i*P_i + C_i* P_{i}^{2}))\\
  \text{subject to} & \\
&\sum_{i=1}^{n} P_i = L \\
& P_{min,i}  \leq P_i y_i \leq P_{max,i}\\
\end{aligned}
$$

Here, $y_i$ is a binary variable representing the state of the i-th generator ; 1 if on, 0 if
off. $A_i,B_i , C_i$ are constants ,  $P_i  \in \mathbb{R}$.

Following [[1]](https://arxiv.org/abs/2110.12624), we convert the constrained equation above into unconstrained one adding some penalty terms:


$$
\begin{aligned}
 min & \sum_{i=1}^{n} y_i C_i(P) = \sum_{i=1}^{n}(y_i(A_i + B_i*P_i + C_i* P_{i}^{2}))\\
  \text{subject to} & \\
&\sum_{i=1}^{n} \lambda_1 * y_i(P_i -L )^{2} \\
&  \lambda_2 * y_i(P_i -  s_{1,i} -  P_{min,i})^{2} \\
&  \lambda_3 * y_i(P_i +  s_{2,i} -  P_{max,i})^{2} \\
\end{aligned}
$$

where $\lambda_1 , \lambda_2 , \lambda_3$ are some empirical constant , $s_{1,i} ,s_{2,i}$ are some penalty terms where $s_{1,i} = P_i -  P_{min,i}$ , $s_{2,i} = P_{min,i} - P_i$.


## Algorithm overview

We use QAOA to handle the problem of whether the unit is one or off and classical optimzer to handle the continuous variables $\gamma_{i}$ , $\beta_{i}$ and $P_{i}$ for $i = 1,2,...,n$. where $n$ is the number of generating units . Our main task then would be to construct the cost layer.For mixer Hamiltonian,  We will just use the default mixer layer with $R_X$ gates.

Let the input binary bitstring represent $y$. The i-th qubit  will represent whether
the i-th generator is on (|1⟩) or off (|0⟩). The function of a cost layer is to take
this input state and add a phase to it, with the phase angle being the cost of the input state
times some $\gamma_{i}$.


In [ ]:
import numpy as np


In [5]:
class Node:

    def __init__(self, A, B, C, minCapacity , maxCapacity):
        self.A = A
        self.B = B
        self.C = C
        self.minCapacity = minCapacity
        self.maxCapacity = maxCapacity
    def cost(p):
        assert p > self.minCapacity , "power should be greater than minimum power"
        assert p < self.maxCapacity, "power should be smaller than maximum power"
        return A + B*p + C * p**2


In [6]:
class Edge:

    def __init__(self, node1, node2, transmission_cost):
        self.node1 = node1
        self.node2 = node2
        self.transmission_cost = transmission_cost


In [26]:

class UCProblem:

    def __init__(self, load , Nodes):
        self.load = load
        self.Nodes = Nodes
        #self.Edges = Edges
        #self.timestep_count = timestep_count
        #self.grid_timesteps = [Grid(lines, nodes, [True for _ in nodes]) for t in timestep_count]

    def compute_cost(self , pCommit):
        total_Cost = 0.0
        for index in range(len(self.Nodes)):
            total_Cost  =+ nodes[index].cost(pCommit[index])
        return  total_Cost   
    def compute_power_distribution():
        return
        #todo

In [27]:
n=int(input("Enter number of nodes: "))
print("Enter ABC for each node: ")
nodes = []
for i in range(0, n):
    a=int(input("Enter a: "))
    b=int(input("Enter b: "))
    c=int(input("Enter c: "))
    minC=int(input("Enter minimum capacity: "))
    maxC=int(input("Enter maximum capacity: "))
    nodes.append(Node(a , b , c , minC , maxC))
    
print("nodes added successfuly!")


Enter number of nodes: 2
Enter ABC for each node: 
Enter a: 2
Enter b: 2
Enter c: 2
Enter minimum capacity: 2
Enter maximum capacity: 2
Enter a: 2
Enter b: 2
Enter c: 2
Enter minimum capacity: 2
Enter maximum capacity: 2
nodes added successfuly!


In [29]:
pCommit = [30,30,30]
load  = 100.0
uc  = UCProblem(load , nodes)
print('total cost=$ %.3f'%(uc.compute_cost(pCommit)))


total cost=$ 1862.000


####  References

- [[1] Adapting Quantum Approximation Optimization Algorithm (QAOA) for Unit Commitment
](https://arxiv.org/abs/2110.12624) by Samantha Koretsky et al.